In [1]:
import os
import copy
import time
import asyncio
import warnings
import logging

from pathlib import Path

import numpy as np
from astropy.io import fits

from scipy import ndimage
from scipy.signal import medfilt
from scipy.ndimage.filters import gaussian_filter
from astropy.modeling import models, fitting


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [7, 6]

#Import CWFS package
from lsst import cwfs
from lsst.cwfs.instrument import Instrument
from lsst.cwfs.algorithm import Algorithm
from lsst.cwfs.image import Image, readFile, aperture2image, showProjection
import lsst.cwfs.plots as plots

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

In [3]:
await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 30 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 27 messages


[[None, None, None, None, None, None, None], None]

In [4]:
# Declare target name
target_name='16Pup'

In [5]:
focus_offset=0.04557208
expTime=40.

In [12]:
for dz in [0.5, 0.7, 0.9, 1.1, 1.3, 1.5]:
    try:
        await cam.cmd_stopLiveView.start()
    except salobj.AckError:
        print("ack error")

    # Intrafocal hexapod setting - offsets -0.5mm from central focus value above

    print('Move to intrafocal position')
    # await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
    #                                              z=hex_Z-dz, u=hex_U, v=hex_V)
    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

    print('Take intrafocal image')
    cam.evt_endReadout.flush()

    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

    intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

    # Extrafocal hexapod setting
    print('Move to extra-focal hexapod position')
    # await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
    #                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

    # r.evt_endReadout.flush()
    print('Take extra-focal image')
    cam.evt_endReadout.flush()
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

    extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

    print(f"============")
    print(f"dz={dz}")
    print(intra_endron.imageName)
    print(extra_endron.imageName)
    print(f"============")

    try:
        await cam.cmd_startLiveView.set_start(expTime=0.5)
    except salobj.AckError:
        print("ack error")

    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
    
    opt = input("Check rotator and press return:")

    if opt == 'q':
        break
        
    # intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
    # extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
    # intra_fname = wget.download(intra_wget_url)
    # print(f"============")
    # print(f'{intra_fname}')
    # extra_fname = wget.download(extra_wget_url)
    # print(f'{extra_fname}')
    # print(f"============")

Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.5
1579925833-16Pup_intra-0-1
1579925882-16Pup_extra-0-1


Check rotator and press return: 


RemoteEvent(ATPtg, 0, mountDataWarning) falling behind; read 56 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 70 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 70 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(GenericCamera, 1, heartbeat) falling behind; read 69 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 70 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 70 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 69 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 71 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 71 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 71 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 71 messages
RemoteEvent(Gen

Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.7
1579926008-16Pup_intra-0-1
1579926057-16Pup_extra-0-1


Check rotator and press return: 


...

Move to intrafocal position


RemoteEvent(ATMCS, 0, azimuthInPosition) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, allAxesInPosition) falling behind; read 12 messages


Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=1.5
1579927046-16Pup_intra-0-1
1579927100-16Pup_extra-0-1


Check rotator and press return: 


RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(GenericCamera, 1, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteEvent(GenericCamera, 1, logMessage) falling behind; read 95 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) fal

In [10]:
for dz in [0.5, 1.0, 1.5]:
    try:
        await cam.cmd_stopLiveView.start()
    except salobj.AckError:
        print("ack error")

    # Intrafocal hexapod setting - offsets -0.5mm from central focus value above

    print('Move to intrafocal position')
    # await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
    #                                              z=hex_Z-dz, u=hex_U, v=hex_V)
    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

    print('Take intrafocal image')
    cam.evt_endReadout.flush()

    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

    intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

    # Extrafocal hexapod setting
    print('Move to extra-focal hexapod position')
    # await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
    #                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

    # r.evt_endReadout.flush()
    print('Take extra-focal image')
    cam.evt_endReadout.flush()
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

    extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

    print(f"============")
    print(f"dz={dz}")
    print(intra_endron.imageName)
    print(extra_endron.imageName)
    print(f"============")

    try:
        await cam.cmd_startLiveView.set_start(expTime=0.5)
    except salobj.AckError:
        print("ack error")

    attcs.athexapod.evt_positionUpdate.flush()
    await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
    
    opt = input("Check rotator and press return:")

    if opt == 'q':
        break
        
    # intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
    # extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
    # intra_fname = wget.download(intra_wget_url)
    # print(f"============")
    # print(f'{intra_fname}')
    # extra_fname = wget.download(extra_wget_url)
    # print(f'{extra_fname}')
    # print(f"============")

In [7]:
np.array([0.5, 0.7, 0.9, 1.1, 1.3, 1.5])*0.041

array([0.0205, 0.0287, 0.0369, 0.0451, 0.0533, 0.0615])

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis='z', offset=0.)